In [1]:
import numpy as np
import functools as ft
import networkx as nx
from graph import process_dataset, transform
from hdc import hdv, bind, bundle, sbundle, ItemMemory, hdvw, hdva, cosim

In [2]:
def init_vertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [3]:
def init_node_mem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.add_vector(f"mem{n}", graph.nodes[n]["mem"])

In [4]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if cosim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if cosim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.add_vector(f"mem{n}_{i}", mem)

In [5]:
def init_graph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.add_vector("G", G)
    return G

In [6]:
def check_edge(G, A, B, threshold):
    return cosim(B, bind([G, A])) > threshold

In [7]:
def node_memory_reconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = node_memory_reconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [8]:
def graph_memory_reconstruction(G, xs, iter=15):
    raise NotImplementedError
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graph_memory_reconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

In [9]:
# visited = dict()
visited = {}


def shortest_path(G, A, B, xs, distance, threshold, memory):
    label, value, sim = memory.cleanup(bind([G, A]))

    if np.array_equiv(A, B):
        return 0, label

    if label in visited and visited[label][0] <= distance:
        return visited[label][1], visited[label][2]

    visited[label] = [distance, 99, "memnill"]

    neighbours = list(filter(lambda x: check_edge(G, A, x, threshold), xs))

    if len(neighbours) == 0:
        return 99, "memnill"

    dis, lab = min(
        list(
            map(
                lambda a: shortest_path(G, a, B, xs, distance + 1, threshold, memory),
                neighbours,
            )
        ),
        key=lambda x: x[0],
    )
    visited[label][1] = dis + 1
    visited[label][2] = label + lab
    return dis + 1, label + lab

---


# Tests


In [10]:
def test_check_edge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = check_edge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [11]:
def test_node_memory_reconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = node_memory_reconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(cosim(graph.nodes[0]["mem"], memsi[0])),
        )

In [12]:
def test_shortest_path():
    mem = ItemMemory()

    g = nx.gnm_random_graph(NODES, EDGES)
    init_vertices(g, DIMENSIONS)
    init_node_mem(g, mem)
    G = init_graph(g, mem)

    for n in g.nodes():
        print(g.edges(n))

    hdvs = list(map(lambda x: g.nodes[x]["hdv"], g.nodes()))
    origin = 0

    for n in g.nodes():
        print(
            f"{n:02} =>",
            "T" if mem.cleanup(bind([G, hdvs[n]]))[0] == f"mem{n}" else "F",
            mem.cleanup(bind([G, hdvs[n]]))[0],
        )

    test_check_edge(G, g, THRESHOLD)

    for m in g.nodes():
        if origin == m:
            continue
        path = nx.shortest_path(g, origin, m)
        oracle = len(path) - 1
        visited = {}
        test = shortest_path(G, hdvs[origin], hdvs[m], hdvs, 0, THRESHOLD, mem)
        testPath = list(map(int, test[1].split("mem")[1:]))
        print(
            f"{origin}_{m:02} =>",
            oracle,
            test[0],
            "T  " if oracle == test[0] else "F X",
            path,
            testPath,
            nx.is_path(g, testPath),
        )

---


# Main


In [13]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 10000, 0.047, 4

In [14]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    init_vertices(graph, DIMENSIONS)
    init_node_mem(graph, memory)
    retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # test_node_memory_reconstruction(G, graph, 1)
    # test_check_edge(G, graph, THRESHOLD)


# main()
test_shortest_path()

[(0, 29), (0, 21), (0, 20), (0, 17), (0, 12), (0, 23), (0, 28), (0, 16), (0, 22), (0, 9), (0, 15), (0, 26)]
[(1, 29), (1, 7), (1, 11), (1, 21), (1, 5), (1, 6), (1, 15), (1, 26), (1, 28), (1, 16)]
[(2, 22), (2, 3), (2, 24), (2, 15), (2, 7), (2, 23), (2, 4), (2, 11), (2, 18), (2, 14), (2, 19), (2, 16), (2, 13)]
[(3, 23), (3, 10), (3, 2), (3, 11), (3, 20), (3, 12), (3, 18), (3, 5), (3, 25), (3, 24), (3, 22), (3, 16), (3, 19), (3, 17), (3, 8), (3, 7)]
[(4, 17), (4, 14), (4, 21), (4, 2), (4, 29), (4, 25), (4, 16), (4, 23), (4, 10), (4, 20)]
[(5, 9), (5, 22), (5, 3), (5, 17), (5, 14), (5, 8), (5, 1), (5, 7), (5, 11)]
[(6, 25), (6, 26), (6, 19), (6, 8), (6, 1), (6, 17), (6, 14), (6, 16)]
[(7, 1), (7, 24), (7, 12), (7, 2), (7, 5), (7, 26), (7, 27), (7, 3)]
[(8, 21), (8, 26), (8, 16), (8, 6), (8, 19), (8, 5), (8, 28), (8, 9), (8, 3), (8, 11), (8, 25), (8, 15), (8, 17)]
[(9, 5), (9, 14), (9, 13), (9, 8), (9, 12), (9, 0), (9, 28), (9, 10), (9, 16)]
[(10, 3), (10, 16), (10, 23), (10, 21), (10, 4),

0_01 => 2 2 T   [0, 29, 1] [0, 15, 1] True
0_02 => 2 2 T   [0, 22, 2] [0, 15, 1] True
0_03 => 2 2 T   [0, 23, 3] [0, 15, 1] True
0_04 => 2 2 T   [0, 17, 4] [0, 15, 1] True
0_05 => 2 2 T   [0, 9, 5] [0, 15, 1] True
0_06 => 2 2 T   [0, 26, 6] [0, 15, 1] True
0_07 => 2 2 T   [0, 12, 7] [0, 15, 1] True
0_08 => 2 2 T   [0, 21, 8] [0, 15, 1] True
0_09 => 1 2 F X [0, 9] [0, 15, 1] True
0_10 => 2 2 T   [0, 16, 10] [0, 15, 1] True
0_11 => 2 2 T   [0, 16, 11] [0, 15, 1] True
0_12 => 1 2 F X [0, 12] [0, 15, 1] True
0_13 => 2 2 T   [0, 9, 13] [0, 15, 1] True
0_14 => 2 2 T   [0, 9, 14] [0, 15, 1] True
0_15 => 1 2 F X [0, 15] [0, 15, 1] True
0_16 => 1 2 F X [0, 16] [0, 15, 1] True
0_17 => 1 2 F X [0, 17] [0, 15, 1] True
0_18 => 2 2 T   [0, 16, 18] [0, 15, 1] True
0_19 => 2 2 T   [0, 28, 19] [0, 15, 1] True
0_20 => 1 2 F X [0, 20] [0, 15, 1] True
0_21 => 1 2 F X [0, 21] [0, 15, 1] True
0_22 => 1 2 F X [0, 22] [0, 15, 1] True
0_23 => 1 2 F X [0, 23] [0, 15, 1] True
0_24 => 2 2 T   [0, 29, 24] [0, 15, 

---
